# Predict protein secondary structure

In this assignment you will predict the secondary structure of proteins based on their amino acid sequences. The [secondary structure of a protein](https://en.wikipedia.org/wiki/Protein_secondary_structure) is the local spatial arrangement of its backbone atoms. It arises from the hydrogen bonds between the backbone atoms and is the building block of proteins' 3D structure. The most common types of secondary structures are alpha helices, beta sheets, and coils. Given a protein's 3D structure, the secondary structure can be determined from the coordinates of its atoms. Depending on the determination method, the secondary structure can be classified into more or fewer types.

Here you will use a dataset of proteins with known secondary structures that were assigned by the [DSSP](https://pdb-redo.eu/dssp/about) method based on their 3D structures. Although DSSP assigns eight types of secondary structures, the assignment simplifies them into three types: helix (including types G, H, and I), strand (including types B and E) and coil (including types T, S, ., and P). The training and test datasets are provided in the text files [train.txt](../../../_static/protein-secondary-structure/train.txt) and [test.txt](../../../_static/protein-secondary-structure/test.txt), respectively. Each line starting with a greater-than symbol `>` contains the name of the protein. In the training dataset, the two lines following the name contain the amino acid sequence and the secondary structure, respectively. In the test dataset, only the amino acid sequence is provided. The amino acid sequence is represented by the one-letter code. The secondary structure is represented as a string of 0 (helix), 1 (strand), and 2 (other).

The task is to train a machine learning model using the training dataset and predict the secondary structure of the proteins using their sequences in the test dataset.

## Model
Different proteins have different numbers of amino acids. If the model were to predict the secondary structure of all amino acids in a protein at once, it would have to handle sequences of different lengths. Although this is possible, this assignment will use a simpler approach. The model will predict the secondary structure of an amino acid based on the amino acids before it and after it. This way, the model will have a fixed-size input and output. Specifically, the model will use as input a sequence of 15 amino acids and predict the secondary structure of the central amino acid. The model will slide this window along the sequence to predict the secondary structure of all amino acids in a protein. For the amino acids near the beginning and end of the sequence where the window extends beyond the sequence, the model will use a special padding token `*` to complete the window. 

As an example, consider the amino acid sequence `SIVAGYEVVGSSSASELLSAIEHVAEKA`. To predict the secondary structure of the starting amino acid `S`, the model will use as input the window `*******SIVAGYEV`. For the second amino acid `I`, the model will use the window `******SIVAGYEVV`. For the residue `A` at position 14, the model will use the window 
`EVVGSSSASELLSAI`. This process will continue until the model predicts the secondary structure of the last amino acid `A` using the window `IEHVAEKA*******`

Therefore the input to the model will be a sequence of 15 amino acids and the output will be the secondary structure of the central amino acid. The secondary structure of an amino acid is a categorical variable $y$ with three classes: 0 (helix), 1 (strand), and 2 (other). Therefore, the model will perform a multi-class classification. The input is a sequence of strings, so they need to be converted to numerical values. The simplest way to do this is to use [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics). In one-hot encoding, each amino acid is represented by a vector of zeros with a one at the position corresponding to the amino acid. For example, if all amino acids (including the padding token `*`) are represented by the list `['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '*']`, then the amino acid `A` is represented by the 21-dimensional vector `[1, 0, ..., 0]`, the amino acid `C` is represented by `[0, 1, 0, ..., 0]`, and so on. The padding token `*` is represented by `[0, 0, ..., 1]`. Because the input is a sequence of 15 amino acids, the input $x$ to the model is a vector of length $15 \times 21 = 315$.

The model will be a multi-class logistic regression model, as covered in the lectures.

In [19]:
import numpy as np
import jax.numpy as jnp
import jax
from tqdm import tqdm
import optax
import chex
import optax.tree_utils as otu
from typing import NamedTuple
jax.config.update("jax_enable_x64", True)

### Process the training data
Given the design of the model, the data in `train.txt` are not directly suitable for training. The data need to be processed to create the input and output pairs for the model. For each sequence in the training data, you need to extract all windows of 15 amino acids and the corresponding secondary structure of the central amino acid. Then you need to convert the amino acids to one-hot encoding. Take the sequence `SIVAGYEVVGSSSASELLSAIEHVAEKA` and its secondary structure `2111111111222000000000220000` as an example. The first window is `*******SIVAGYEV` and the corresponding secondary structure of the central amino acid `S` is 2. The second window is `******SIVAGYEVV` and the secondary structure of the central amino acid `I` is 1. This process will continue until the last window `IEHVAEKA*******` and the secondary structure of the central amino acid `A` is 0. Each window and the corresponding secondary structure will be an input-output pair for the model. You need to process all sequences in the training data to create the input-output pairs.

In [20]:
path_to_train = '../data/train.txt'


## read data from train.txt
train_seq = {}

## finish the code below such that train_seq is a dictionary with the following structure:
## train_seq[protein_name] = (sequence, secondary_structure), 
## where protein_name is the name of the protein, sequence is the amino acid sequence of the protein, and secondary_structure is the secondary structure of the protein as given in the train.txt file


with open(path_to_train, 'r') as f:
    ############################################
    #### write your code (5 points) #######
    ############################################
    for line in f:
        if line.startswith(">"):
            name = line.strip()[1:]
            train_seq[name] = []
        else:
            train_seq[name].append(line.strip())


In [21]:
def get_windows(seq, secondary_structure):
    """ Extract windows of size 15 from the sequence and secondary structure of a protein

    Args:
        seq: str, the amino acid sequence of the protein
        secondary_structure: str, the secondary structure of the protein

    Returns:
        list of tuples: each tuple contains (x, y), where x (15x21=315 dimensional) is the one-hot encoding of the amino acid sequence of length 15, and y is the secondary structure of the central amino acid in the window
    """

    ## amino acid order for one-hot encoding
    amino_acids = "ACDEFGHIKLMNPQRSTVWY*"

    ############################################
    #### write your code (5 points) ############
    ############################################

    def one_hot_encoding(residue):
        return np.array([1 if residue == aa else 0 for aa in amino_acids], dtype=np.uint8)

    windows = []
    for i, r in enumerate(seq):
        if i < 7:
            segment = "*"*(7-i) + seq[:i+8]
        elif i > len(seq) - 8:
            segment = seq[i-7:] + "*"*(7-(len(seq)-1-i))
        else:
            segment = seq[i-7:i+8]        

        x = np.array([one_hot_encoding(aa) for aa in segment])
        x = x.reshape(-1)
        y = secondary_structure[i]

        windows.append((x, y))

    return windows

Code in the cell below uses the function `get_windows` to process every sequence in the training data and stack both `x` and `y` in numpy arrays: `train_xs` and `train_y`.

In [22]:
#############################################################################
#### The following code is provided to you and you need to understand it ####

## get windows for all proteins in the training set
## stack all windows in train_xs and train_ys

train_xs = []
train_ys = []

for name, (seq, ss) in tqdm(train_seq.items()):
    windows = get_windows(seq, ss)
    for x, y in windows:
        train_xs.append(x)
        train_ys.append(y)

train_xs = np.array(train_xs)
train_ys = np.array(train_ys, dtype=np.int64)

train_xs = np.hstack((np.ones((train_xs.shape[0], 1)), train_xs))

assert train_xs.shape[0] == train_ys.shape[0]
assert train_xs.shape[1] == 315 + 1
assert np.all((train_xs == 0) | (train_xs == 1))

100%|██████████| 4776/4776 [00:43<00:00, 109.62it/s]


Code in this cell builds a optimizer, `fmin_lbfgs`, using the `optax` library. You do not need to modify this code. The optimization step will use this optimizer.

In [23]:
def _run_opt(init_params, fun, opt, max_iter, tol):
    value_and_grad_fun = optax.value_and_grad_from_state(fun)

    def step(carry):
        params, state = carry
        value, grad = value_and_grad_fun(params, state=state)
        updates, state = opt.update(
            grad, state, params, value=value, grad=grad, value_fn=fun
        )
        params = optax.apply_updates(params, updates)
        return params, state

    def continuing_criterion(carry):
        _, state = carry
        iter_num = otu.tree_get(state, "count")
        grad = otu.tree_get(state, "grad")
        err = otu.tree_l2_norm(grad)
        return (iter_num == 0) | ((iter_num < max_iter) & (err >= tol))

    init_carry = (init_params, opt.init(init_params))
    final_params, final_state = jax.lax.while_loop(
        continuing_criterion, step, init_carry
    )
    return final_params, final_state


class _InfoState(NamedTuple):
    iter_num: chex.Numeric


def _print_info():
    def init_fn(params):
        del params
        return _InfoState(iter_num=0)

    def update_fn(updates, state, params, *, value, grad, **extra_args):
        del params, extra_args

        jax.debug.print(
            "Iteration: {i:>5}, Value: {v:>6.3e}, Gradient norm: {e:>6.3e}",
            i=state.iter_num,
            v=value,
            e=otu.tree_l2_norm(grad),
        )
        return updates, _InfoState(iter_num=state.iter_num + 1)

    return optax.GradientTransformationExtraArgs(init_fn, update_fn)

def fmin_lbfgs(initial_params, fun, max_iter, tol):
    opt = optax.chain(_print_info(), optax.lbfgs())
    return _run_opt(initial_params, fun, opt, max_iter, tol)

## Define the loss function for softmax regression
In the following, you need to complete a few functions to compute the loss for the softmax regression model.
The parameter of the loss function `loss_fn` is `theta` which has shape `(2, 316)`, although we have three classes. The first row of `theta` corresponds to the weights for the class 0 and the second row corresponds to the weights for the class 1. Why do we not have a row for class 2? This is because the degenaracy of the softmax function, i.e., the softmax function is invariant to adding a constant to the scores. Therefore, we can fix one of the scores to zero. In this case, we fix the score of the last class to zero by setting the `theta` parameter for the last class to zero. This is why the shape of `theta` is `(2, 316)` and not `(3, 316)`. Given a `theta` of shape `(2, 316)`, we could always add a row of zeros to make it `(3, 316)`.

In [24]:
def expand_theta(theta):
    """Expand theta by appending a row of zeros at the end

    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)

    Returns:
        jnp.ndarray: the expanded theta with shape (3, 316) where the last row is all zeros

    """

    return jnp.vstack((theta, jnp.zeros_like(theta[0])))


def log_likelihood_per_sample(theta, x, y):
    """Compute the log-likelihood of a single sample

    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)
        x: jnp.ndarray, the input features of one sample
        y: int, the target label of the sample

    Returns:
        float: the log-likelihood of the sample

    """

    theta_expanded = expand_theta(theta)
    logit = jnp.matmul(theta_expanded, x)
    logprob = logit - jax.scipy.special.logsumexp(logit)
    return logprob[y]


def log_likelihood_data(theta, xs, ys):
    """Compute the log-likelihood of the data by using log_likelihood_per_sample and jax.vmap

    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)
        xs: jnp.ndarray, the input features of all samples with shape (n, 316)
        ys: jnp.ndarray, the target labels of all samples with shape (n,)

    Returns:
        jnp.ndarray: the log-likelihood of all samples with shape (n,)

    """

    return jnp.sum(
        jax.vmap(log_likelihood_per_sample, in_axes=(None, 0, 0))(theta, xs, ys)
    )


def loss_fn(theta):
    """the loss function is defined as the mean negative log-likelihood of the samples,

    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)

    Returns:
        float: the mean negative log-likelihood of the samples

    """

    return -log_likelihood_data(theta, train_xs, train_ys) / train_xs.shape[0]

## Train the model
After the data are processed properly and the loss function is defined, it is time to train the model. Thanks to the automatic differentiation provided by JAX, you do not need to write the function to compute the gradient of the loss function. You only need to provide the loss function to the optimizer along with the initial parameters and stopping criteria. 

In [25]:
## initialize theta with zeros
m = 3
theta_init = jnp.zeros((m-1, train_xs.shape[1]))

## check the initial value of the loss function and the norm of the gradient at the initial point
print(
    f'Initial value: {loss_fn(theta_init):.2e} '
    f'Initial gradient norm: {otu.tree_l2_norm(jax.grad(loss_fn)(theta_init)):.2e}'
)

## optimize the loss function using L-BFGS 
final_theta, _ = fmin_lbfgs(theta_init, loss_fn, max_iter=300, tol=1e-4)
print(
    f'Final value: {loss_fn(final_theta):.2e}, '
    f'Final gradient norm: {otu.tree_l2_norm(jax.grad(loss_fn)(final_theta)):.2e}'
)

Initial value: 1.10e+00 Initial gradient norm: 1.93e-01
Iteration:     0, Value: 1.099e+00, Gradient norm: 1.934e-01
Iteration:     1, Value: 1.070e+00, Gradient norm: 1.148e-01
Iteration:     2, Value: 1.040e+00, Gradient norm: 8.036e-02
Iteration:     3, Value: 1.013e+00, Gradient norm: 8.489e-02
Iteration:     4, Value: 9.647e-01, Gradient norm: 1.042e-01
Iteration:     5, Value: 9.065e-01, Gradient norm: 6.281e-02
Iteration:     6, Value: 8.863e-01, Gradient norm: 3.913e-02
Iteration:     7, Value: 8.799e-01, Gradient norm: 2.339e-02
Iteration:     8, Value: 8.691e-01, Gradient norm: 4.872e-02
Iteration:     9, Value: 8.601e-01, Gradient norm: 5.991e-02
Iteration:    10, Value: 8.491e-01, Gradient norm: 2.996e-02
Iteration:    11, Value: 8.465e-01, Gradient norm: 8.120e-03
Iteration:    12, Value: 8.458e-01, Gradient norm: 1.338e-02
Iteration:    13, Value: 8.450e-01, Gradient norm: 1.955e-02
Iteration:    14, Value: 8.439e-01, Gradient norm: 1.869e-02
Iteration:    15, Value: 8.43

## Make predictions and compute the accuracy on the training data
After the model is trained, you need to make predictions on the training data and compute the accuracy of the predictions. For each residue in the training data, you need to compute the probability of each class and assign the class with the highest probability as the predicted class. Then you need to compare the predicted classes with the true classes and compute the accuracy. The accuracy is the number of correct predictions divided by the total number of predictions.

In [26]:
def predict_per_sample(theta, x):
    """Predict the class of a single sample
    
    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)
        x: jnp.ndarray, the input features of one sample

    Returns:
        int: the predicted class of the sample
    
    """
    theta = expand_theta(theta)
    return jnp.argmax(jnp.matmul(theta, x))

def predict(theta, xs):
    """Predict the classes of multiple samples by using predict_per_sample and jax.vmap

    Args:
        theta: jnp.ndarray, the parameters of the model with shape (2, 316)
        xs: jnp.ndarray, the input features of multiple samples with shape (n, 316)

    Returns:
        jnp.ndarray: the predicted classes of the samples with shape (n,)        
    """

    return jax.vmap(predict_per_sample, in_axes=(None, 0))(theta, xs)


def compute_accuray(y_pred, y_true):
    """Compute the accuracy of the model

    Args:
        y_pred: jnp.ndarray, the predicted classes of the samples with shape (n,)
        y_true: jnp.ndarray, the true classes of the samples with shape (n,)

    Returns:
        float: the accuracy of the model
    """

    return jnp.mean(y_pred == y_true)


y_pred = predict(final_theta, train_xs)

Compute the accuracy of the model on the training set using the initial theta and the final theta. The accuracy with the initial theta should be around 33% because the initial theta is zero and the model is making random predictions. The accuracy with the final theta should be significantly higher because the model has learned the weights that minimize the loss function on the training data.

In [27]:
y_pred_init = predict(theta_init, train_xs)
accuracy_init = compute_accuray(y_pred_init, train_ys)

y_pred_final = predict(final_theta, train_xs)
accuracy_final = compute_accuray(y_pred_final, train_ys)

print(f'Accuracy of the model with initial theta: {accuracy_init:.2f}')
print(f'Accuracy of the model with final theta: {accuracy_final:.2f}')


Accuracy of the model with initial theta: 0.38
Accuracy of the model with final theta: 0.62


## Make predictions on the test data
After the model is trained, you need to make predictions on the test sequences. The test sequences are provided in the file `test.txt`. You need to use the trained model to predict the secondary structure of all residues in the test sequences. The predictions should be written to a text file named `predictions.txt` and its format should be the same as the file `train.txt`.
